In [ ]:
dayObs = 20250514  # format YYYYMMDD, e.g. "20250514"
bin_number = 100

# SITCOM-2089 Evaluate M1M3 hard points individual forces (histogram)

This notebook makes a histogram of maximum forces and spread of all slews passing certain conditions given a day_obs.

### Prepare Notebook

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from astropy.time import Time
import matplotlib.dates as mdates

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient
from lsst_efd_client import EfdClient

import warnings
warnings.filterwarnings("ignore")

from lsst.ts.xml.enums.MTM1M3 import DetailedStates

In [ ]:
# create a client to retrieve datasets in the EFD database
client = EfdClient("usdf_efd")

In [ ]:
OPERATIONAL_LIMIT = 450 # newtons,TBC with Petr
SAFE_LIMIT = 900
BREAKAWAY_LIMIT = 3200

### Define functions

#### get previous logged detailedState event from a given time stamp

The status of M1M3 is not persistent in the EFD. In order to get at a given time what is the current status, use this function.

In [ ]:
def get_previous_logged_detailedState(df_state, timestamp):
    """
    Get logged detailedState from M1M3 immediately before arbitrary time
    Args:
       df_state (pandas dataframe): pandas dataframe obtained from  time series of
          "lsst.sal.MTM1M3.logevent_detailedState" covering a wide time frame which includes
          the time stamp
       timestamp (pandas timestamp): a timestamp where we want to probe the current status of M1M3
    Returns:
       prev_state: human readable status of current M1M3 status
    """
    df_state_names = df_state["detailedState"].map(lambda x: DetailedStates(x).name)
    previous_index = df_state.index.asof(timestamp)
    try:
        prev = df_state.index.get_loc(previous_index)
    except KeyError:
        return "KeyError"
    return df_state_names[prev]

### Define relevant settings

#### Observation day

In [ ]:
## Insert here the day_obs of interest
day_obs = dayObs

### Load data

In [ ]:
# Select data from a given date
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(day_obs)

# Get lists of slew and track events
slews = [e for e in events if e.type == TMAState.SLEWING]
tracks = [e for e in events if e.type == TMAState.TRACKING]
print(f"There are {len(events)} events")
print(f"Found {len(slews)} slews and {len(tracks)} tracks")

### Select slews passing certain criteria

In [ ]:
slews_selected = []
hp_max_hist = np.array([])
hp_spread_max_hist = np.array([])
min_ele_range = -1  # minimum elevation change in slew, degrees
min_azi_range = -1  # minimum azimuth change in slew, degrees
hp_threshold = OPERATIONAL_LIMIT
velocity_threshold = 0.5 # deg/s

df_state = getEfdData(
    client,
    "lsst.sal.MTM1M3.logevent_detailedState",
    begin=Time(slews[0].begin, format="isot", scale="utc"),
    end=Time(slews[-1].end, format="isot", scale="utc"),
)  # get an array for all state changes from first slew of day_obs to final one

for i, slew in enumerate(slews):
    if (
        slew.seqNum == 0
    ):  # skip first one to avoid problems looking for a previous detailedState outside the df_state range
        continue
        
    df_azi = getEfdData(client, "lsst.sal.MTMount.azimuth", event=slew)
    df_ele = getEfdData(client, "lsst.sal.MTMount.elevation", event=slew)
    df_hp = getEfdData(client, "lsst.sal.MTM1M3.hardpointActuatorData", event=slew)
    timestamp = pd.Timestamp(
        Time(slew.begin, format="iso", scale="utc").value, tz="utc"
    )
    begin_state = get_previous_logged_detailedState(df_state, timestamp)
    
    if begin_state == "KeyError":
        continue
        
    timestamp = pd.Timestamp(Time(slew.end, format="iso", scale="utc").value, tz="utc")
    end_state = get_previous_logged_detailedState(df_state, timestamp)
    
    if len(df_hp) > 0: 
        hp_max_individual = np.array(
            [
                np.max(abs(df_hp["measuredForce0"].values)),
                np.max(abs(df_hp["measuredForce1"].values)),
                np.max(abs(df_hp["measuredForce2"].values)),
                np.max(abs(df_hp["measuredForce3"].values)),
                np.max(abs(df_hp["measuredForce4"].values)),
                np.max(abs(df_hp["measuredForce5"].values)),
            ]
        )
        hp_max = np.max(hp_max_individual)
        hp_spread_individual = np.array(
            [
                np.max(df_hp["measuredForce0"].values)-np.min(df_hp["measuredForce0"].values),
                np.max(df_hp["measuredForce1"].values)-np.min(df_hp["measuredForce1"].values),
                np.max(df_hp["measuredForce2"].values)-np.min(df_hp["measuredForce2"].values),
                np.max(df_hp["measuredForce3"].values)-np.min(df_hp["measuredForce3"].values),
                np.max(df_hp["measuredForce4"].values)-np.min(df_hp["measuredForce4"].values),
                np.max(df_hp["measuredForce5"].values)-np.min(df_hp["measuredForce5"].values),
            ]
        )
        hp_spread_max = np.max(hp_spread_individual)
        
        # introduce conditions to select specific slews

        # condition on minimum azimuth and minimum elevation
        slew_delta_azi = df_azi["demandPosition"].max() - df_azi["demandPosition"].min()
        slew_delta_ele = df_ele["demandPosition"].max() - df_ele["demandPosition"].min()
        slew_ele_condition = slew_delta_ele > min_ele_range
        slew_azi_condition = slew_delta_azi > min_azi_range

        # ensure that the HPs are active (mirror is 'raised')
        state_condition = (  
            (begin_state == "ACTIVE") or (begin_state == "ACTIVEENGINEERING")
        ) and ((end_state == "ACTIVE") or (end_state == "ACTIVEENGINEERING"))

        # condition on maximum azimuth velocity
        velocity_condition = (df_azi["actualVelocity"] < velocity_threshold).all()

        # check all conditions to select slews and fill histogram
        if (
            slew_ele_condition
            and slew_azi_condition
            and state_condition
            and velocity_condition
        ):
            hp_max_hist = np.append(hp_max_hist,hp_max) 
            hp_spread_max_hist = np.append(hp_spread_max_hist,hp_spread_max)


### Plot histograms

#### Plot distribution of maximum forces

In [ ]:
# Plot a distribution of maximum forces
plt.ylabel("Number of slews")
plt.xlabel("Maximum recorded force in any hardpoint per slew (N)")
plt.title(f"Maximum force on hardpoints for {day_obs}")
plt.axvline(x=OPERATIONAL_LIMIT, color='orange', linestyle='--', linewidth=2, label='Operational limit')
plt.axvline(x=SAFE_LIMIT, color='red', linestyle='--', linewidth=2, label='Safe limit')
plt.axvline(x=BREAKAWAY_LIMIT, color='black', linestyle='--', linewidth=2, label='Breakaway limit')
plt.legend()
h = plt.hist(hp_max_hist, bins=bin_number)

#### Plot distribution of difference of maximum and minimum

In [ ]:
plt.ylabel("Number of slews")
plt.xlabel("Maximum spread of forces in any hardpoint per slew (N)")
plt.title(f"Maximum spread of forces on hardpoints for {day_obs}")
plt.legend()
h = plt.hist(hp_spread_max_hist, bins=bin_number)